In [1]:
import pandas as pd
import sqlite3 
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [3]:
c.execute("""
            SELECT Div, HomeTeam, 
            SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as h_win,
            SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as h_draw,
            SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as h_loss
            FROM Matches
            WHERE Season = 2011
            GROUP BY HomeTeam
            ORDER BY h_win DESC
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(10)

,Div,HomeTeam,h_win,h_draw,h_loss
0,E0,Man City,18,1,0
1,E0,Man United,15,2,2
2,D1,Bayern Munich,14,1,2
3,D1,Dortmund,14,2,1
4,D2,Greuther Furth,14,1,2
5,D1,Schalke 04,13,1,3
6,E0,Tottenham,13,3,3
7,E0,Arsenal,12,4,3
8,E0,Chelsea,12,3,4
9,D2,St Pauli,12,3,2


In [8]:
c.execute("""
            SELECT Div, AwayTeam, 
            SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as a_win,
            SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as a_draw,
            SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as a_loss
            FROM Matches
            WHERE Season = 2011
            GROUP BY AwayTeam
            ORDER BY a_win DESC
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(10)

,Div,AwayTeam,a_win,a_draw,a_loss
0,E0,Man United,13,3,3
1,D1,Dortmund,11,4,2
2,E0,Man City,10,4,5
3,E0,Arsenal,9,3,7
4,D1,Bayern Munich,9,3,5
5,D2,Ein Frankfurt,9,3,5
6,E0,Liverpool,8,1,10
7,D1,M'gladbach,8,2,7
8,E0,Newcastle,8,3,8
9,D2,Paderborn,8,4,5


In [3]:
c.execute("""SELECT 
                    HomeTeam as team,
                    h_win,
                    a_win,
                    h_draw,
                    a_draw,
                    h_loss,
                    a_loss
            
            FROM
                
            (SELECT Div, HomeTeam, 
            SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as h_win,
            SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as h_draw,
            SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as h_loss
            FROM Matches
            WHERE Season = 2011
            GROUP BY HomeTeam)
            
            JOIN
            
            (SELECT AwayTeam, 
            SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as a_win,
            SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as a_draw,
            SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as a_loss
            FROM Matches
            WHERE Season = 2011
            GROUP BY AwayTeam)
            
            ON (HomeTeam==AwayTeam)
            
            WHERE Div='E0'
            
            ORDER BY h_win DESC
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(5)

,team,h_win,a_win,h_draw,a_draw,h_loss,a_loss
0,Man City,18,10,1,4,0,5
1,Man United,15,13,2,3,2,3
2,Tottenham,13,7,3,6,3,6
3,Arsenal,12,9,4,3,3,7
4,Chelsea,12,6,3,7,4,6


In [27]:
c.execute("""SELECT 
                    HomeTeam as Club,
                    (h_win + a_win + h_draw + a_draw + h_loss + a_loss) as MP,
                    (h_win + a_win) as W,
                    (h_draw + a_draw) as D,
                    (h_loss + a_loss) as L,
                    (h_goals_for+a_goals_for) as GF,
                    (h_goals_agst+a_goals_agst) as GA,
                    (h_goals_for+a_goals_for-h_goals_agst-a_goals_agst) as GD,
                    ((h_win + a_win)*3+(h_draw + a_draw)) as Pts
                    
            FROM
                
            (SELECT Div, HomeTeam, 
                    SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as h_win,
                    SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as h_draw,
                    SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as h_loss,
                    SUM(FTHG) as h_goals_for,
                    SUM(FTAG) as h_goals_agst
            FROM Matches
            WHERE Season = 1995
            GROUP BY HomeTeam
            ORDER BY HomeTeam)
            
            JOIN
            
            (SELECT AwayTeam, 
                    SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as a_win,
                    SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as a_draw,
                    SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as a_loss,
                    SUM(FTAG) as a_goals_for,
                    SUM(FTHG) as a_goals_agst
            FROM Matches
            WHERE Season = 1995
            GROUP BY AwayTeam
            ORDER BY AwayTeam)
            
            ON (HomeTeam==AwayTeam)
            
            WHERE Div='E0'
            
            ORDER BY Pts DESC, GD DESC, GF DESC
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,Club,MP,W,D,L,GF,GA,GD,Pts
0,Man United,38,25,7,6,73,35,38,82
1,Newcastle,38,24,6,8,66,37,29,78
2,Liverpool,38,20,11,7,70,34,36,71
3,Aston Villa,38,18,9,11,52,35,17,63
4,Arsenal,38,17,12,9,49,32,17,63
5,Everton,38,17,10,11,64,44,20,61
6,Blackburn,38,18,7,13,61,47,14,61
7,Tottenham,38,16,13,9,50,38,12,61
8,Nott'm Forest,38,15,13,10,50,54,-4,58
9,West Ham,38,14,9,15,43,52,-9,51


In [2]:
c.execute("""SELECT * FROM Matches;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.sample(n=10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
8594,30742,D2,1993,1994-06-04,Saarbrucken,Meppen,1,1,D
1751,1752,D2,2012,2012-09-26,Hertha,Dresden,1,0,H
1565,1566,D2,2011,2011-08-28,Munich 1860,Union Berlin,3,1,H
2753,2754,D2,2013,2014-02-14,Fortuna Dusseldorf,Union Berlin,1,1,D
23095,45245,E0,2012,2013-05-19,Chelsea,Everton,2,1,H
12067,34215,D2,1999,1999-11-05,St Pauli,Chemnitz,0,0,D
23121,45271,E0,2013,2013-08-24,Southampton,Sunderland,1,1,D
11616,33764,D2,1998,1999-05-07,F Koln,Ulm,2,2,D
738,739,D1,2010,2010-08-28,Schalke 04,Hannover,1,2,A
6636,6637,D1,2007,2007-11-24,Leverkusen,Duisburg,4,1,H


In [28]:
#list(df['Season'].unique())

In [29]:
#df.info()

In [5]:
df_2011 = df.loc[df['Season']==2011]

In [6]:
c.execute("""SELECT 
                    HomeTeam as Club,
                    (h_win + a_win + h_draw + a_draw + h_loss + a_loss) as MP,
                    (h_win + a_win) as W,
                    (h_draw + a_draw) as D,
                    (h_loss + a_loss) as L,
                    (h_goals_for+a_goals_for) as GF,
                    (h_goals_agst+a_goals_agst) as GA,
                    (h_goals_for+a_goals_for-h_goals_agst-a_goals_agst) as GD,
                    ((h_win + a_win)*3+(h_draw + a_draw)) as Pts
                    
            FROM
                
            (SELECT Div, HomeTeam, 
                    SUM(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) as h_win,
                    SUM(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) as h_draw,
                    SUM(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) as h_loss,
                    SUM(FTHG) as h_goals_for,
                    SUM(FTAG) as h_goals_agst
            FROM Matches
            WHERE Season = 2012
            GROUP BY HomeTeam
            ORDER BY HomeTeam)
            
            JOIN
            
            (SELECT AwayTeam, 
                    SUM(CASE WHEN FTAG > FTHG THEN 1 ELSE 0 END) as a_win,
                    SUM(CASE WHEN FTAG = FTHG THEN 1 ELSE 0 END) as a_draw,
                    SUM(CASE WHEN FTAG < FTHG THEN 1 ELSE 0 END) as a_loss,
                    SUM(FTAG) as a_goals_for,
                    SUM(FTHG) as a_goals_agst
            FROM Matches
            WHERE Season = 2012
            GROUP BY AwayTeam
            ORDER BY AwayTeam)
            
            ON (HomeTeam==AwayTeam)
            
            WHERE Div='E0'
            
            ORDER BY Pts DESC, GD DESC, GF DESC
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.index = df.index + 1
df

,Club,MP,W,D,L,GF,GA,GD,Pts
1,Man United,38,28,5,5,86,43,43,89
2,Man City,38,23,9,6,66,34,32,78
3,Chelsea,38,22,9,7,75,39,36,75
4,Arsenal,38,21,10,7,72,37,35,73
5,Tottenham,38,21,9,8,66,46,20,72
6,Everton,38,16,15,7,55,40,15,63
7,Liverpool,38,16,13,9,71,43,28,61
8,West Brom,38,14,7,17,53,57,-4,49
9,Swansea,38,11,13,14,47,51,-4,46
10,West Ham,38,12,10,16,45,53,-8,46


In [6]:
df_2011.head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
1096,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
1097,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D
1098,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A
1099,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A
1100,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H


In [7]:
df_2011.shape

(992, 9)

In [12]:
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, SUM(FTHG) as hgoals FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hgoals_df = pd.DataFrame(c.fetchall())
hgoals_df.columns = [x[0] for x in c.description]
hgoals_df.head()

,Unique_Team_ID,Div,HomeTeam,hgoals
0,1,D1,Bayern Munich,49
1,2,D1,Dortmund,44
2,3,D1,Leverkusen,28
3,5,D1,Schalke 04,47
4,6,D1,M'gladbach,29


In [38]:
#hgoals_df['HomeTeam'].unique

In [13]:
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, SUM(FTAG) as agoals FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011
                GROUP BY AwayTeam
                ORDER BY AwayTeam;""")
agoals_df = pd.DataFrame(c.fetchall())
agoals_df.columns = [x[0] for x in c.description]
agoals_df.head()

,Unique_Team_ID,Div,AwayTeam,agoals
0,34,D2,Aachen,15
1,81,E0,Arsenal,35
2,82,E0,Aston Villa,17
3,15,D1,Augsburg,16
4,1,D1,Bayern Munich,28


In [39]:
#agoals_df['AwayTeam'].unique

In [14]:
#team home+away goals
goals_df = pd.merge(hgoals_df, agoals_df, on='Unique_Team_ID')

In [15]:
goals_df['total_goals'] = goals_df['hgoals'] + goals_df['agoals']

In [16]:
goals_df = goals_df.drop(['Div_y','AwayTeam'], axis=1)

In [17]:
goals_df.head()

,Unique_Team_ID,Div_x,HomeTeam,hgoals,agoals,total_goals
0,1,D1,Bayern Munich,49,28,77
1,2,D1,Dortmund,44,36,80
2,3,D1,Leverkusen,28,24,52
3,5,D1,Schalke 04,47,27,74
4,6,D1,M'gladbach,29,20,49


In [18]:
#home wins
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, COUNT(FTR) as hwins FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011 AND FTR is 'H'
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hwins_df = pd.DataFrame(c.fetchall())
hwins_df.columns = [x[0] for x in c.description]
hwins_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins
0,1,D1,Bayern Munich,14
1,2,D1,Dortmund,14
2,3,D1,Leverkusen,8
3,5,D1,Schalke 04,13
4,6,D1,M'gladbach,9


In [19]:
hwins_df.shape

(56, 4)

In [20]:
#away wins
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, COUNT(FTR) as awins FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011 AND FTR is 'A'
                GROUP BY AwayTeam
                ORDER BY Unique_Team_ID;""")
awins_df = pd.DataFrame(c.fetchall())
awins_df.columns = [x[0] for x in c.description]
awins_df.head()

,Unique_Team_ID,Div,AwayTeam,awins
0,1,D1,Bayern Munich,9
1,2,D1,Dortmund,11
2,3,D1,Leverkusen,7
3,5,D1,Schalke 04,7
4,6,D1,M'gladbach,8


In [21]:
awins_df.shape

(56, 4)

In [22]:
#home losses
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, COUNT(FTR) as hloss FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011 AND FTR is 'A'
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hloss_df = pd.DataFrame(c.fetchall())
hloss_df.columns = [x[0] for x in c.description]
hloss_df.head()

,Unique_Team_ID,Div,HomeTeam,hloss
0,1,D1,Bayern Munich,2
1,2,D1,Dortmund,1
2,3,D1,Leverkusen,5
3,5,D1,Schalke 04,3
4,6,D1,M'gladbach,1


In [23]:
hloss_df.shape

(54, 4)

In [24]:
#away losses
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, COUNT(FTR) as aloss FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011 AND FTR is 'H'
                GROUP BY AwayTeam
                ORDER BY Unique_Team_ID;""")
aloss_df = pd.DataFrame(c.fetchall())
aloss_df.columns = [x[0] for x in c.description]
aloss_df.head()

,Unique_Team_ID,Div,AwayTeam,aloss
0,1,D1,Bayern Munich,5
1,2,D1,Dortmund,2
2,3,D1,Leverkusen,5
3,5,D1,Schalke 04,7
4,6,D1,M'gladbach,7


In [25]:
#home draws
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, COUNT(FTR) as hdraw FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011 AND FTR is 'D'
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hdraw_df = pd.DataFrame(c.fetchall())
hdraw_df.columns = [x[0] for x in c.description]
hdraw_df.head()

,Unique_Team_ID,Div,HomeTeam,hdraw
0,1,D1,Bayern Munich,1
1,2,D1,Dortmund,2
2,3,D1,Leverkusen,4
3,5,D1,Schalke 04,1
4,6,D1,M'gladbach,7


In [26]:
#away draws
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, COUNT(FTR) as adraw FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011 AND FTR is 'D'
                GROUP BY AwayTeam
                ORDER BY Unique_Team_ID;""")
adraw_df = pd.DataFrame(c.fetchall())
adraw_df.columns = [x[0] for x in c.description]
adraw_df.head()

,Unique_Team_ID,Div,AwayTeam,adraw
0,1,D1,Bayern Munich,3
1,2,D1,Dortmund,4
2,3,D1,Leverkusen,5
3,5,D1,Schalke 04,3
4,6,D1,M'gladbach,2


In [74]:
#combine home and away wins

In [27]:
#merge wins
record_df = pd.merge(hwins_df, awins_df[['Unique_Team_ID', 'awins']], on='Unique_Team_ID')

In [28]:
#merge losses
record_df = pd.merge(record_df, hloss_df[['Unique_Team_ID', 'hloss']], on='Unique_Team_ID', how='left')

In [29]:
record_df = pd.merge(record_df, aloss_df[['Unique_Team_ID', 'aloss']], on='Unique_Team_ID', how='left')

In [30]:
#merge draws
record_df = pd.merge(record_df, hdraw_df[['Unique_Team_ID', 'hdraw']], on='Unique_Team_ID', how='left')

In [31]:
record_df = pd.merge(record_df, adraw_df[['Unique_Team_ID', 'adraw']], on='Unique_Team_ID', how='left')

In [32]:
record_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins,awins,hloss,aloss,hdraw,adraw
0,1,D1,Bayern Munich,14,9,2.0,5,1,3
1,2,D1,Dortmund,14,11,1.0,2,2,4
2,3,D1,Leverkusen,8,7,5.0,5,4,5
3,5,D1,Schalke 04,13,7,3.0,7,1,3
4,6,D1,M'gladbach,9,8,1.0,7,7,2


In [33]:
record_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 9 columns):
Unique_Team_ID    56 non-null int64
Div               56 non-null object
HomeTeam          56 non-null object
hwins             56 non-null int64
awins             56 non-null int64
hloss             54 non-null float64
aloss             56 non-null int64
hdraw             56 non-null int64
adraw             56 non-null int64
dtypes: float64(1), int64(6), object(2)
memory usage: 4.4+ KB


In [34]:
record_df['hloss'] = record_df['hloss'].apply(lambda x: 0 if pd.isnull(x) else int(x))

In [35]:
record_df['total_matches'] = (record_df['hwins'] + record_df['awins'] + record_df['hloss'] + record_df['aloss'] + 
                             record_df['hdraw'] + record_df['adraw'])

In [36]:
record_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins,awins,hloss,aloss,hdraw,adraw,total_matches
0,1,D1,Bayern Munich,14,9,2,5,1,3,34
1,2,D1,Dortmund,14,11,1,2,2,4,34
2,3,D1,Leverkusen,8,7,5,5,4,5,34
3,5,D1,Schalke 04,13,7,3,7,1,3,34
4,6,D1,M'gladbach,9,8,1,7,7,2,34


In [37]:
df_2011['Div'].value_counts()

E0    380
D1    306
D2    306
Name: Div, dtype: int64

In [38]:
record_df['tot_wins'] = record_df['hwins']+record_df['awins']
record_df['tot_draws'] = record_df['hdraw']+record_df['adraw']
record_df['tot_losses'] = record_df['hloss']+record_df['aloss']

In [39]:
record_df['TeamName'] = record_df['HomeTeam']

In [40]:
record_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins,awins,hloss,aloss,hdraw,adraw,total_matches,tot_wins,tot_draws,tot_losses,TeamName
0,1,D1,Bayern Munich,14,9,2,5,1,3,34,23,4,7,Bayern Munich
1,2,D1,Dortmund,14,11,1,2,2,4,34,25,6,3,Dortmund
2,3,D1,Leverkusen,8,7,5,5,4,5,34,15,9,10,Leverkusen
3,5,D1,Schalke 04,13,7,3,7,1,3,34,20,4,10,Schalke 04
4,6,D1,M'gladbach,9,8,1,7,7,2,34,17,9,8,M'gladbach


In [41]:
record_df = record_df[['Unique_Team_ID', 'Div','TeamName', 'tot_wins', 'tot_draws', 'tot_losses', 'total_matches']]

### Consolidated Records Table

In [42]:
record_df.head(10)

,Unique_Team_ID,Div,TeamName,tot_wins,tot_draws,tot_losses,total_matches
0,1,D1,Bayern Munich,23,4,7,34
1,2,D1,Dortmund,25,6,3,34
2,3,D1,Leverkusen,15,9,10,34
3,5,D1,Schalke 04,20,4,10,34
4,6,D1,M'gladbach,17,9,8,34
5,7,D1,Wolfsburg,13,5,16,34
6,8,D1,FC Koln,8,6,20,34
7,9,D1,Hoffenheim,10,11,13,34
8,10,D1,Hertha,7,10,17,34
9,11,D1,Mainz,9,12,13,34


In [43]:
df_2011.loc[df_2011['Div'] == 'E0'].head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
22345,44495,E0,2011,2011-08-13,Blackburn,Wolves,1,2,A
22346,44496,E0,2011,2011-08-13,Fulham,Aston Villa,0,0,D
22347,44497,E0,2011,2011-08-13,Liverpool,Sunderland,1,1,D
22348,44498,E0,2011,2011-08-13,Newcastle,Arsenal,0,0,D
22349,44499,E0,2011,2011-08-13,QPR,Bolton,0,4,A


In [44]:
c.execute("""SELECT * FROM Teams_in_Matches;""")
match_df = pd.DataFrame(c.fetchall())
match_df.columns = [x[0] for x in c.description]
match_df.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [45]:
c.execute("""SELECT * FROM Unique_Teams;""")
team_df = pd.DataFrame(c.fetchall())
team_df.columns = [x[0] for x in c.description]
team_df.head()

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [46]:
df_2011.head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
1096,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
1097,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D
1098,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A
1099,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A
1100,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H


In [48]:
goals_df2 = goals_df[['Unique_Team_ID', 'total_goals']]

In [51]:
record_df = pd.merge(record_df, goals_df2, on='Unique_Team_ID')

In [53]:
record_df.head(10)

,Unique_Team_ID,Div,TeamName,tot_wins,tot_draws,tot_losses,total_matches,total_goals
0,1,D1,Bayern Munich,23,4,7,34,77
1,2,D1,Dortmund,25,6,3,34,80
2,3,D1,Leverkusen,15,9,10,34,52
3,5,D1,Schalke 04,20,4,10,34,74
4,6,D1,M'gladbach,17,9,8,34,49
5,7,D1,Wolfsburg,13,5,16,34,47
6,8,D1,FC Koln,8,6,20,34,39
7,9,D1,Hoffenheim,10,11,13,34,41
8,10,D1,Hertha,7,10,17,34,38
9,11,D1,Mainz,9,12,13,34,47


In [55]:
record_df.loc[record_df['Div']=='E0'].head(10)

,Unique_Team_ID,Div,TeamName,tot_wins,tot_draws,tot_losses,total_matches,total_goals
36,81,E0,Arsenal,21,7,10,38,74
37,82,E0,Aston Villa,7,17,14,38,37
38,83,E0,Chelsea,18,10,10,38,65
39,84,E0,Liverpool,14,10,14,38,47
40,85,E0,Man City,28,5,5,38,93
41,86,E0,Newcastle,19,8,11,38,56
42,91,E0,Norwich,12,11,15,38,52
43,92,E0,Tottenham,20,9,9,38,66
44,93,E0,Everton,15,11,12,38,50
45,97,E0,Blackburn,8,7,23,38,48


In [21]:
#pseudocode for conceptualization
c.execute("""
           SELECT Div, HomeTeam, 
                    SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as h_win,
                    SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as h_draw,
                    SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as h_loss,
                    SUM(FTHG) as h_goals_for,
                    SUM(FTAG) as h_goals_agst
            FROM Matches
            WHERE Season = 2012
            GROUP BY HomeTeam
            ORDER BY HomeTeam
            
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(10)

,Div,HomeTeam,h_win,h_draw,h_loss,h_goals_for,h_goals_agst
0,D2,Aalen,7,4,6,24,17
1,E0,Arsenal,11,5,3,47,23
2,E0,Aston Villa,5,5,9,23,28
3,D1,Augsburg,5,5,7,19,22
4,D1,Bayern Munich,14,2,1,56,11
5,D2,Bochum,5,4,8,24,25
6,D2,Braunschweig,10,6,1,29,14
7,E0,Chelsea,12,5,2,41,16
8,D2,Cottbus,9,5,3,28,15
9,D1,Dortmund,10,3,4,40,19


In [23]:
c.execute("""
            SELECT Div, AwayTeam, 
                    SUM(CASE WHEN FTR is 'A' THEN 1 ELSE 0 END) as a_win,
                    SUM(CASE WHEN FTR is 'D' THEN 1 ELSE 0 END) as a_draw,
                    SUM(CASE WHEN FTR is 'H' THEN 1 ELSE 0 END) as a_loss,
                    SUM(FTAG) as a_goals_for,
                    SUM(FTHG) as a_goals_agst
            FROM Matches
            WHERE Season = 2012
            GROUP BY AwayTeam
            ORDER BY AwayTeam
            
            
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(10)

,Div,AwayTeam,a_win,a_draw,a_loss,a_goals_for,a_goals_agst
0,D2,Aalen,5,6,6,16,22
1,E0,Arsenal,10,5,4,25,14
2,E0,Aston Villa,5,6,8,24,41
3,D1,Augsburg,3,4,10,14,29
4,D1,Bayern Munich,15,2,0,42,7
5,D2,Bochum,5,4,8,16,27
6,D2,Braunschweig,9,4,4,23,20
7,E0,Chelsea,10,4,5,34,23
8,D2,Cottbus,3,7,7,13,21
9,D1,Dortmund,9,6,2,41,23


In [26]:
c.execute("""SELECT *
            FROM
            (SELECT Div, HomeTeam, 
                    SUM(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) as h_win,
                    SUM(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) as h_draw,
                    SUM(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) as h_loss,
                    SUM(FTHG) as h_goals_for,
                    SUM(FTAG) as h_goals_agst
            FROM Matches
            WHERE Season = 2011
            GROUP BY HomeTeam
            ORDER BY HomeTeam)
            
            JOIN
            
            (SELECT AwayTeam, 
                    SUM(CASE WHEN FTAG > FTHG THEN 1 ELSE 0 END) as a_win,
                    SUM(CASE WHEN FTAG = FTHG THEN 1 ELSE 0 END) as a_draw,
                    SUM(CASE WHEN FTAG < FTHG THEN 1 ELSE 0 END) as a_loss,
                    SUM(FTAG) as a_goals_for,
                    SUM(FTHG) as a_goals_agst
            FROM Matches
            WHERE Season = 2011
            GROUP BY AwayTeam
            ORDER BY AwayTeam)
            
            ON (HomeTeam==AwayTeam)
            
            
            
        """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(10)

,Div,HomeTeam,h_win,h_draw,h_loss,h_goals_for,h_goals_agst,AwayTeam,a_win,a_draw,a_loss,a_goals_for,a_goals_agst
0,D2,Aachen,4,6,7,15,24,Aachen,2,7,8,15,23
1,E0,Arsenal,12,4,3,39,17,Arsenal,9,3,7,35,32
2,E0,Aston Villa,4,7,8,20,25,Aston Villa,3,10,6,17,28
3,D1,Augsburg,6,7,4,20,19,Augsburg,2,7,8,16,30
4,D1,Bayern Munich,14,1,2,49,6,Bayern Munich,9,3,5,28,16
5,E0,Blackburn,6,1,12,26,33,Blackburn,2,6,11,22,45
6,D2,Bochum,7,3,7,23,23,Bochum,3,4,10,18,32
7,E0,Bolton,4,4,11,23,39,Bolton,6,2,11,23,38
8,D2,Braunschweig,6,8,3,21,15,Braunschweig,4,7,6,16,20
9,E0,Chelsea,12,3,4,41,24,Chelsea,6,7,6,24,22
